# Laboratorio 3.5: Bloc de notas del estudiante

## Información general

Este laboratorio es una continuación de los laboratorios guiados del Módulo 3. 

En este laboratorio, implementará un modelo entrenado y realizará una predicción en relación con el modelo. A continuación, eliminará el punto de enlace y realizará una transformación por lotes en el conjunto de datos de prueba.


## Introducción al escenario empresarial

Usted trabaja para un proveedor de atención médica y desea mejorar la detección de anomalías en pacientes ortopédicos. 

Tiene la tarea de resolver este problema mediante el aprendizaje automático. Tiene acceso a un conjunto de datos que contiene seis características biomecánicas y un objetivo de *normal* o *anormal*. Puede utilizar este conjunto de datos para entrenar un modelo de aprendizaje automático para predecir si un paciente va a tener una anomalía.


## Acerca de este conjunto de datos

Este conjunto de datos biomédicos fue creado por el Dr. Henrique da Mota durante un período de residencia médica en el Grupo de Investigación Aplicada en Ortopedia (GARO, Group of Applied Research in Orthopaedics) del Centre Médico-Chiurgical de Réadaptation des Massues, Lyon, Francia. Los datos se han organizado en dos tareas de clasificación diferentes, pero relacionadas. 

La primera tarea consiste en clasificar a los pacientes como pertenecientes a una de tres categorías: 

- *Normal* (100 pacientes)
- *Hernia de disco* (60 pacientes)
- *Espondilolistesis* (150 pacientes)

Para la segunda tarea, las categorías *Hernia de disco* y *Espondilolistesis* se fusionaron en una sola categoría etiquetada como *anormal*. Por lo tanto, la segunda tarea consiste en clasificar a los pacientes como pertenecientes a una de dos categorías: *Normal* (100 pacientes) o *Anormal* (210 pacientes).


## Información de atributos

Cada paciente está representado en el conjunto de datos por seis atributos biomecánicos que se derivan de la forma y orientación de la pelvis y la columna lumbar (en este orden): 

- Incidencia pélvica
- Inclinación pélvica
- Ángulo de lordosis lumbar
- Inclinación del sacro
- Radio pélvico
- Grado de espondilolistesis

La siguiente convención se utiliza para las etiquetas de clase: 
- Hernia Disco (HD)
- Espondilolistesis (EL)
- Normal (NO) 
- Anormal (AN)

Para obtener más información acerca de este conjunto de datos, consulte [Página web del conjunto de datos de la columna vertebral] (http://archive.ics.uci.edu/ml/datasets/Vertebral+Column).


## Atribuciones del conjunto de datos

Este conjunto de datos se obtuvo de:
Dua, D. y Graff, C. (2019). Repositorio de aprendizaje automático de la UCI (http://archive.ics.uci.edu/ml). Irvine, CA: Universidad de California, Escuela de Ciencias de la Información e Informática.


# Configuración del laboratorio

Dado que esta solución se divide en varios laboratorios del módulo, debe ejecutar las siguientes celdas para poder cargar los datos y entrenar el modelo que se desea implementar.

**Nota:** La configuración puede tardar hasta 5 minutos en completarse.

## Importación de los datos

Al ejecutar las celdas siguientes, los datos se importarán y estarán listos para su uso. 

**Nota:** Las siguientes celdas representan los pasos clave en los laboratorios anteriores.


In [1]:
bucket='c141604a3635652l8512213t1w041906854371-labbucket-7mrq3fg5xnrz'

In [2]:
import warnings, requests, zipfile, io
warnings.simplefilter('ignore')
import pandas as pd
from scipy.io import arff

import os
import boto3
import sagemaker
from sagemaker.image_uris import retrieve
from sklearn.model_selection import train_test_split

[11/24/24 09:43:17] INFO     Found credentials from IAM Role:                                   ]8;id=612097;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=560239;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1075\1075]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [4]:
f_zip = 'http://archive.ics.uci.edu/ml/machine-learning-databases/00212/vertebral_column_data.zip'
r = requests.get(f_zip, stream=True)
Vertebral_zip = zipfile.ZipFile(io.BytesIO(r.content))
Vertebral_zip.extractall()

data = arff.loadarff('column_2C_weka.arff')
df = pd.DataFrame(data[0])

class_mapper = {b'Abnormal':1,b'Normal':0}
df['class']=df['class'].replace(class_mapper)

cols = df.columns.tolist()
cols = cols[-1:] + cols[:-1]
df = df[cols]

train, test_and_validate = train_test_split(df, test_size=0.2, random_state=42, stratify=df['class'])
test, validate = train_test_split(test_and_validate, test_size=0.5, random_state=42, stratify=test_and_validate['class'])

prefix='lab3'

train_file='vertebral_train.csv'
test_file='vertebral_test.csv'
validate_file='vertebral_validate.csv'

s3_resource = boto3.Session().resource('s3')
def upload_s3_csv(filename, folder, dataframe):
    csv_buffer = io.StringIO()
    dataframe.to_csv(csv_buffer, header=False, index=False )
    s3_resource.Bucket(bucket).Object(os.path.join(prefix, folder, filename)).put(Body=csv_buffer.getvalue())

upload_s3_csv(train_file, 'train', train)
upload_s3_csv(test_file, 'test', test)
upload_s3_csv(validate_file, 'validate', validate)

container = retrieve('xgboost',boto3.Session().region_name,'1.0-1')

hyperparams={"num_round":"42",
             "eval_metric": "auc",
             "objective": "binary:logistic"}

s3_output_location="s3://{}/{}/output/".format(bucket,prefix)
xgb_model=sagemaker.estimator.Estimator(container,
                                       sagemaker.get_execution_role(),
                                       instance_count=1,
                                       instance_type='ml.m4.xlarge',
                                       output_path=s3_output_location,
                                        hyperparameters=hyperparams,
                                        sagemaker_session=sagemaker.Session())

train_channel = sagemaker.inputs.TrainingInput(
    "s3://{}/{}/train/".format(bucket,prefix,train_file),
    content_type='text/csv')

validate_channel = sagemaker.inputs.TrainingInput(
    "s3://{}/{}/validate/".format(bucket,prefix,validate_file),
    content_type='text/csv')

data_channels = {'train': train_channel, 'validation': validate_channel}

xgb_model.fit(inputs=data_channels, logs=False)

print('ready for hosting!')

[11/24/24 09:50:12] INFO     Found credentials from IAM Role:                                   ]8;id=445640;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=441901;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1075\1075]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

[11/24/24 09:50:13] INFO     Defaulting to only available Python version: py3                     ]8;id=486338;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=858420;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/image_uris.py#603\603]8;;\

                    INFO     Defaulting to only supported image scope: cpu.                       ]8;id=569767;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=851404;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/image_uris.py#527\527]8;;\

                    INFO     Creating training-job with name:                                       ]8;id=488317;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=892545;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#1042\1042]8;;\
                             sagemaker-xgboost-2024-11-24-09-50-13-429                                             


2024-11-24 09:50:13 Starting - Starting the training job....
2024-11-24 09:50:41 Starting - Preparing the instances for training....
2024-11-24 09:51:08 Downloading - Downloading input data.....
2024-11-24 09:51:38 Downloading - Downloading the training image.........
2024-11-24 09:52:29 Training - Training image download completed. Training in progress...
2024-11-24 09:52:44 Uploading - Uploading generated training model..
2024-11-24 09:52:57 Completed - Training job completed
ready for hosting!


# Paso 1: Alojar el modelo

Ahora que tiene un modelo entrenado, puede alojarlo utilizando los servicios de alojamiento de Amazon SageMaker.

El primer paso es implementar el modelo. Debido a que tiene un objeto modelo, *modelo_xgb*, puede utilizar el método **implementar**. Para este laboratorio, utilizará una única instancia ml.m4.xlarge.



In [5]:
xgb_predictor = xgb_model.deploy(initial_instance_count=1,
                serializer = sagemaker.serializers.CSVSerializer(),
                instance_type='ml.m4.xlarge')

[11/24/24 09:53:57] INFO     Creating model with name: sagemaker-xgboost-2024-11-24-09-53-57-202    ]8;id=598183;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=772114;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#4025\4025]8;;\

                    INFO     Creating endpoint-config with name                                     ]8;id=654601;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=922882;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#5820\5820]8;;\
                             sagemaker-xgboost-2024-11-24-09-53-57-202                                             

[11/24/24 09:53:58] INFO     Creating endpoint with name sagemaker-xgboost-2024-11-24-09-53-57-202  ]8;id=228500;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=503666;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#4642\4642]8;;\

-------!

# Paso 2: Realizar predicciones

Ahora que tiene un modelo implementado, deberá ejecutar algunas predicciones.

Primero, revise los datos de prueba y vuelva a familiarizarse con ellos.

In [6]:
test.shape

(31, 7)

Tiene 31 instancias, con siete atributos. Las cinco primeras instancias son:

In [7]:
test.head(5)

,class,pelvic_incidence,pelvic_tilt,lumbar_lordosis_angle,sacral_slope,pelvic_radius,degree_spondylolisthesis
136,1,88.024499,39.844669,81.774473,48.179830,116.601538,56.766083
230,0,65.611802,23.137919,62.582179,42.473883,124.128001,-4.083298
134,1,52.204693,17.212673,78.094969,34.992020,136.972517,54.939134
130,1,50.066786,9.120340,32.168463,40.946446,99.712453,26.766697
47,1,41.352504,16.577364,30.706191,24.775141,113.266675,-4.497958


No es necesario incluir el valor objetivo (clase). Este predictor puede tomar datos en el formato de valores separados por comas (CSV). Por lo tanto, puede obtener la primera fila *sin la columna de clase* utilizando el siguiente código:

`test.iloc[:1,1:]` 

La función **iloc** toma parámetros de [*filas*, *colum*]

Para obtener solo la primera fila, use `0:1`. Si desea obtener la fila 2, podría usar `1:2`.

Para obtener todas las columnas *excepto* la primera columna (*colum 0*), use `1:`



In [14]:
row = test.iloc[1:2,1:]
row.head()

,pelvic_incidence,pelvic_tilt,lumbar_lordosis_angle,sacral_slope,pelvic_radius,degree_spondylolisthesis
230,65.611802,23.137919,62.582179,42.473883,124.128001,-4.083298


Puede convertir esto en un archivo de valores separados por comas (CSV) y almacenarlo en un búfer de cadena.

In [15]:
batch_X_csv_buffer = io.StringIO()
row.to_csv(batch_X_csv_buffer, header=False, index=False)
test_row = batch_X_csv_buffer.getvalue()
print(test_row)

65.61180231,23.13791922,62.58217893,42.47388309,124.1280012,-4.083298414



Ahora, puede usar los datos para realizar una predicción.

In [16]:
xgb_predictor.predict(test_row)

b'0.777283251285553'

El resultado que obtiene no es un *0* o un *1*. En cambio, obtendrá una *puntuación de probabilidad*. Puede aplicar alguna lógica condicional a la puntuación de probabilidad para determinar si la respuesta debe presentarse como un 0 o un 1. Trabajará con este proceso cuando realice predicciones por lotes.

Por ahora, compare el resultado con los datos de prueba.

In [17]:
test.head(5)

,class,pelvic_incidence,pelvic_tilt,lumbar_lordosis_angle,sacral_slope,pelvic_radius,degree_spondylolisthesis
136,1,88.024499,39.844669,81.774473,48.179830,116.601538,56.766083
230,0,65.611802,23.137919,62.582179,42.473883,124.128001,-4.083298
134,1,52.204693,17.212673,78.094969,34.992020,136.972517,54.939134
130,1,50.066786,9.120340,32.168463,40.946446,99.712453,26.766697
47,1,41.352504,16.577364,30.706191,24.775141,113.266675,-4.497958


**Pregunta:** ¿Es exacta la predicción?

**Tarea de desafío:** Actualice el código anterior para enviar la segunda fila del conjunto de datos. ¿Son correctas esas predicciones? Pruebe esta tarea con algunas otras filas.

Puede ser tedioso enviar estas filas de una a la vez. Puede escribir una función para enviar estos valores en un lote, pero SageMaker ya tiene una capacidad por lotes. Examinará esa característica a continuación. Sin embargo, antes de hacerlo, deberá terminar el modelo.

# Paso 3: Terminar el modelo implementado

Para eliminar el punto de enlace, utilice la función **eliminar_punto_de_enlace** en el predictor.

In [18]:
xgb_predictor.delete_endpoint(delete_endpoint_config=True)

[11/24/24 10:01:17] INFO     Deleting endpoint configuration with name:                             ]8;id=636499;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=139734;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#4796\4796]8;;\
                             sagemaker-xgboost-2024-11-24-09-53-57-202                                             

                    INFO     Deleting endpoint with name: sagemaker-xgboost-2024-11-24-09-53-57-202 ]8;id=846385;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=240448;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#4786\4786]8;;\

# Paso 4: Realizar una transformación por lotes

Cuando se encuentra en el ciclo de diseño de características de entrenamiento-prueba, desea probar sus conjuntos de retención o de pruebas con el modelo. A continuación, puede utilizar esos resultados para calcular métricas. Puede implementar un punto de enlace como lo hizo anteriormente, pero después debe recordar eliminar el punto de enlace. Sin embargo, hay una manera más eficiente.

Puede utilizar el método transformador del modelo para obtener un objeto transformador. A continuación, puede utilizar el método de transformación de este objeto para realizar una predicción en todo el conjunto de datos de prueba. SageMaker: 

- Activará una instancia con el modelo
- Realizará una predicción sobre todos los valores de entrada
- Escribirá esos valores en Amazon Simple Storage Service (Amazon S3) 
Por último, finalizará la instancia

Comenzará convirtiendo sus datos en un archivo CSV que el objeto transformador puede tomar como entrada. Esta vez, usará **iloc** para obtener todas las filas y todas las columnas *excepto* la primera columna.


In [19]:
batch_X = test.iloc[:,1:];
batch_X.head()

,pelvic_incidence,pelvic_tilt,lumbar_lordosis_angle,sacral_slope,pelvic_radius,degree_spondylolisthesis
136,88.024499,39.844669,81.774473,48.179830,116.601538,56.766083
230,65.611802,23.137919,62.582179,42.473883,124.128001,-4.083298
134,52.204693,17.212673,78.094969,34.992020,136.972517,54.939134
130,50.066786,9.120340,32.168463,40.946446,99.712453,26.766697
47,41.352504,16.577364,30.706191,24.775141,113.266675,-4.497958


A continuación, escriba sus datos en un archivo CSV.

In [20]:
batch_X_file='batch-in.csv'
upload_s3_csv(batch_X_file, 'batch-in', batch_X)

Por último, antes de realizar una transformación, configure el transformador con el archivo de entrada, la ubicación de salida y el tipo de instancia.

In [21]:
batch_output = "s3://{}/{}/batch-out/".format(bucket,prefix)
batch_input = "s3://{}/{}/batch-in/{}".format(bucket,prefix,batch_X_file)

xgb_transformer = xgb_model.transformer(instance_count=1,
                                       instance_type='ml.m4.xlarge',
                                       strategy='MultiRecord',
                                       assemble_with='Line',
                                       output_path=batch_output)

xgb_transformer.transform(data=batch_input,
                         data_type='S3Prefix',
                         content_type='text/csv',
                         split_type='Line')
xgb_transformer.wait()

[11/24/24 10:01:49] INFO     Creating model with name: sagemaker-xgboost-2024-11-24-10-01-49-137    ]8;id=350761;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=508524;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#4025\4025]8;;\

                    INFO     Creating transform job with name:                                      ]8;id=570329;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=818427;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#3882\3882]8;;\
                             sagemaker-xgboost-2024-11-24-10-01-49-730                                             

....................................
[2024-11-24:10:07:53:INFO] No GPUs detected (normal if no gpus installed)
[2024-11-24:10:07:53:INFO] No GPUs detected (normal if no gpus installed)
[2024-11-24:10:07:53:INFO] nginx config: 
worker_processes auto;
daemon off;
pid /tmp/nginx.pid;
error_log  /dev/stderr;
worker_rlimit_nofile 4096;
events {
  worker_connections 2048;
}
http {
  include /etc/nginx/mime.types;
  default_type application/octet-stream;
  access_log /dev/stdout combined;
  upstream gunicorn {
    server unix:/tmp/gunicorn.sock;
  }
  server {
    listen 8080 deferred;
    client_max_body_size 0;
    keepalive_timeout 3;
    location ~ ^/(ping|invocations|execution-parameters) {
      proxy_set_header X-Forwarded-For $proxy_add_x_forwarded_for;
      proxy_set_header Host $http_host;
      proxy_redirect off;
      proxy_read_timeout 60s;
      proxy_pass http://gunicorn;
    }
    location / {
      return 404 "{}";
    }
  }
}
[2024-11-24 10:07:53 +0000] [19] [INFO] Startin

Una vez finalizada la transformación, puede descargar los resultados de Amazon S3 y compararlos con la entrada.

Primero, descargue la salida de Amazon S3 y cárguela en un DataFrame de pandas.


In [22]:
s3 = boto3.client('s3')
obj = s3.get_object(Bucket=bucket, Key="{}/batch-out/{}".format(prefix,'batch-in.csv.out'))
target_predicted = pd.read_csv(io.BytesIO(obj['Body'].read()),',',names=['class'])
target_predicted.head(5)

,class
0,0.996607
1,0.777283
2,0.994641
3,0.993690
4,0.939139


Puede utilizar una función para convertir la probabilidad en un *0* o un *1*.

La primera salida de la tabla serán los *valores previstos*, y la segunda salida de la tabla son los *datos de prueba originales*.

In [29]:
def binary_convert(x):
    threshold = 0.65
    if x > threshold:
        return 1
    else:
        return 0

target_predicted['binary'] = target_predicted['class'].apply(binary_convert)

print(target_predicted.head(10))
test.head(10)

      class  binary
0  0.996607       1
1  0.777283       1
2  0.994641       1
3  0.993690       1
4  0.939139       1
5  0.997396       1
6  0.991977       1
7  0.987518       1
8  0.993334       1
9  0.682776       1


,class,pelvic_incidence,pelvic_tilt,lumbar_lordosis_angle,sacral_slope,pelvic_radius,degree_spondylolisthesis
136,1,88.024499,39.844669,81.774473,48.179830,116.601538,56.766083
230,0,65.611802,23.137919,62.582179,42.473883,124.128001,-4.083298
134,1,52.204693,17.212673,78.094969,34.992020,136.972517,54.939134
130,1,50.066786,9.120340,32.168463,40.946446,99.712453,26.766697
47,1,41.352504,16.577364,30.706191,24.775141,113.266675,-4.497958
135,1,77.121344,30.349874,77.481083,46.771470,110.611148,82.093607
100,1,84.585607,30.361685,65.479486,54.223922,108.010218,25.118478
89,1,71.186811,23.896201,43.696665,47.290610,119.864938,27.283985
297,0,45.575482,18.759135,33.774143,26.816347,116.797007,3.131910
4,1,49.712859,9.652075,28.317406,40.060784,108.168725,7.918501


**Nota:** El *umbral* de la función **conversión_binaria** se establece en *0,65*.

**Tarea de desafío:** Experimente con el cambio del valor del umbral. ¿Afecta los resultados?

**Nota:** El modelo inicial puede no ser bueno. Generará algunas métricas en el siguiente laboratorio, antes de ajustar el modelo en el laboratorio final.

# ¡Felicitaciones!

Ha completado este laboratorio y ahora puede terminarlo siguiendo las instrucciones de la guía del laboratorio.